In [1]:
import sleepy
import pandas as pd
import numpy as np
import random
import itertools
from tqdm import tqdm
import time
import math
import plotly.express as px
import copy

In [2]:
USERNAME = 'TheRealFergus'
YEAR = 2023
N = 1000000 # The number of similutaions to run

In [4]:
# Gets league and owner data
user_id = sleepy.get_user_data(USERNAME)["user_id"]
league_id = sleepy.get_league_ids(user_id, YEAR)[0]
league_raw = sleepy.get_leage(league_id)
owners_raw = sleepy.get_owners(league_id)
owners = owners_raw[["username", "owner_id", "roster_id"]]

In [5]:
owners

,username,owner_id,roster_id
0,alecwilson,781258862778015744,1
1,namebrant,737201118836346880,2
2,therealfergus,871830995287085056,3
3,empireyikesback,340376049508429824,4
4,pacc,791907251894984704,5
5,tonygordzilla22,790423754491678720,6
6,mackjyers21,463115290251620352,7
7,burgertownthicnred,865421962913157120,8
8,thezirconisdragon,865438032692649984,9
9,black8yellownation,865844843182694400,10


In [16]:
# Stores the number of teams in the league
num_teams = owners.shape[0]

# Stores the number of the first week of the league playoffs
playoff_week1 = league_raw["settings"]["playoff_week_start"]

# Maps a unique matchup id corresponding to each combination of two roster ids
matchup_to_roster_id = {matchup[0] + 1: matchup[1] for matchup in 
                        enumerate(itertools.combinations(range(1,13), 2))}

roster_to_matchup_id = {val: key for (key, val) in matchup_to_roster_id.items()}

In [45]:
# Gets regular season matchup data
matchups = (sleepy.get_matchups(league_id, season=True)
            .query(f"starter == True & week < {playoff_week1}")
            .groupby(["week", "roster_id", "matchup_id"])
            [["team_points"]]
            .first()
            .reset_index())

# Merges matchups with owners to include usersernames
matchups = (matchups.merge(owners.reset_index(), left_on="roster_id", 
                                                 right_on = "roster_id")
                    .assign(matchup_id = matchups["matchup_id"].astype(int)))

matchups["matchup_id"] = matchups.groupby(["week", "matchup_id"])["roster_id"].transform(lambda x: roster_to_matchup_id[tuple(x.unique())])
matchups.head(12)

,week,roster_id,matchup_id,team_points,index,username,owner_id
0,1,1,10,95.92,0,alecwilson,781258862778015744
1,1,2,15,102.31,1,namebrant,737201118836346880
2,1,3,27,87.10,2,therealfergus,871830995287085056
3,1,4,36,80.88,3,empireyikesback,340376049508429824
4,1,5,40,75.52,4,pacc,791907251894984704
5,1,6,15,104.94,5,tonygordzilla22,790423754491678720
6,1,7,40,116.85,6,mackjyers21,463115290251620352
7,1,8,60,105.44,7,burgertownthicnred,865421962913157120
8,1,9,27,138.10,8,thezirconisdragon,865438032692649984
9,1,10,36,87.78,9,black8yellownation,865844843182694400


In [56]:
# Formats the actual season schedule
season_schedule = [tuple(week) for week in 
                   matchups.groupby("week")["matchup_id"].agg(set)]

# Gets the total season points for each owner
owners_points = matchups.groupby("username")[["team_points"]].sum()

In [68]:
# Gets all possible weekly schedules
all_weeks = []
for week in tqdm(itertools.combinations(matchup_to_roster_id.values(), 6), 
                 total = math.comb(66, 6)):
    s = set()
    for match in week:
        s.update(match)
    if len(s) == 12:
        all_weeks.append([roster_to_matchup_id[match] for match in week])

100%|██████████| 90858768/90858768 [00:44<00:00, 2060985.59it/s]


In [67]:
len(all_weeks)

[(3, 7, 11, 15, 19, 23),
 (3, 7, 11, 15, 20, 22),
 (3, 7, 11, 15, 21, 21),
 (3, 7, 11, 16, 18, 23),
 (3, 7, 11, 16, 19, 22),
 (3, 7, 11, 16, 20, 21),
 (3, 7, 11, 17, 17, 23),
 (3, 7, 11, 17, 19, 21),
 (3, 7, 11, 17, 20, 20),
 (3, 7, 11, 18, 17, 22),
 (3, 7, 11, 18, 18, 21),
 (3, 7, 11, 18, 20, 19),
 (3, 7, 11, 19, 17, 21),
 (3, 7, 11, 19, 18, 20),
 (3, 7, 11, 19, 19, 19),
 (3, 7, 12, 14, 19, 23),
 (3, 7, 12, 14, 20, 22),
 (3, 7, 12, 14, 21, 21),
 (3, 7, 12, 15, 18, 23),
 (3, 7, 12, 15, 19, 22),
 (3, 7, 12, 15, 20, 21),
 (3, 7, 12, 16, 17, 23),
 (3, 7, 12, 16, 19, 21),
 (3, 7, 12, 16, 20, 20),
 (3, 7, 12, 17, 17, 22),
 (3, 7, 12, 17, 18, 21),
 (3, 7, 12, 17, 20, 19),
 (3, 7, 12, 18, 17, 21),
 (3, 7, 12, 18, 18, 20),
 (3, 7, 12, 18, 19, 19),
 (3, 7, 13, 13, 19, 23),
 (3, 7, 13, 13, 20, 22),
 (3, 7, 13, 13, 21, 21),
 (3, 7, 13, 15, 17, 23),
 (3, 7, 13, 15, 18, 22),
 (3, 7, 13, 15, 19, 21),
 (3, 7, 13, 16, 16, 23),
 (3, 7, 13, 16, 18, 21),
 (3, 7, 13, 16, 19, 20),
 (3, 7, 13, 17, 16, 22),


In [57]:
# Maps each week schedule in all possible weeks to a set of weeks. Given that the
# key week appears in a season schedule, none of the weeks in the value set can 
# also appear
similar_weeks = {}
for key_week in tqdm(all_weeks, total = len(all_weeks)):
    similar_weeks[key_week] = set([week for week in all_weeks if len(set(week + key_week)) != 12])

100%|██████████| 10395/10395 [00:23<00:00, 442.68it/s]


In [62]:
def generate_schedule():
    """Generates a 14 week season schedule for an 12 team leage. Each team plays
    all other teams once in the first 11 weeks. The first 3 weeks are repeated
    for the last 3 weeks.

    Returns:
        list: A length 14 list of length 6 tuples containing the matchup ids for
    each week.
    """
    weeks = set(all_weeks)
    schedule = []

    # Recursively fills the weekly schedule at random
    def pick_week(weeks):
        if len(schedule) == 11:
            return
        else:
            tuple_weeks = tuple(weeks)
            choice = random.choice(tuple_weeks)
            schedule.append(choice)
            weeks -= similar_weeks[choice]
            pick_week(weeks)
    
    # The above method has about a 70% success rate to pick a valid yearly 
    # schedule. This while loop will continue until a valid schedule is picked.
    while len(schedule) != 11:
        try:
            pick_week(weeks)
        except IndexError:
            weeks = set(all_weeks)
            schedule = []

    # Adds the first three weeks of the schedule to the end to finish the 14
    # week season
    schedule += schedule[:3]
 
    return(schedule)

In [63]:
def get_ranking(schedule):
    """Gets the regular season rankings of the team given a season schedule

    Args:
        schedule (list): A list of tuples containing the matchup ids for each
        week.

    Returns:
        pd.Series: a pandas series indexed by username containing the rank of 
        each team at the end of the season.
    """

    standings = {username: pd.Series([0,0,0], index=["win", "loss", "draw"]) 
             for username in owners["username"]}

    # Records the outcome of a single match in the standings dictionary
    def record_outcome(week_num, matchup_id):

        teams = matchup_to_roster_id[matchup_id]

        df = (
            matchups[(matchups["week"] == week_num) 
                     & ((matchups["roster_id"] == teams[0]) 
                        | (matchups["roster_id"] == teams[1]))]
            )
        
        if df["team_points"].nunique() == 1:
            standings[df.iloc[0, 1]]["draw"] += 1
            standings[df.iloc[1, 1]]["draw"] += 1
        
        else:
            max_username = df.loc[df["team_points"].idxmax(), "username"]
            min_username = df.loc[df["team_points"].idxmin(), "username"]

            standings[max_username]["win"] += 1
            standings[min_username]["loss"] += 1
        
    # Records the outcome of all matches in the standings
    for week_num in range(1, len(schedule) + 1):
        for matchup_id in schedule[week_num - 1]:
            record_outcome(week_num, matchup_id)

    df = (pd.DataFrame.from_dict(standings, orient='index')
          .merge(owners_points, left_index = True, right_index= True)
          .sort_values(by=["win", "team_points"], ascending = False)
          .assign(rank = np.arange(1,13)))

    return df["rank"]


In [64]:
# Stores the true season ranks
season_ranks = get_ranking(season_schedule)
pd.DataFrame(season_ranks)

,rank
pacc,1
thezirconisdragon,2
herbietime,3
alecwilson,4
empireyikesback,5
burgertownthicnred,6
therealfergus,7
mackjyers21,8
shakylegs,9
tonygordzilla22,10


In [65]:
# Initializes a dicitonary to hold simulated rankings
total_records = {username: pd.Series([0 for i in range(12)], index=range(1,13)) for username in owners["username"]}

# Initializes a dictionary to hold playoff seeds
playoff_seeds = {playoff_seed: 0 for playoff_seed in itertools.permutations(owners["username"], 6)}

In [66]:
for i in tqdm(range(N), total = N):

    # Generate a schedule and get the ranking
    ranks = get_ranking(generate_schedule())

    # Store each user's rank
    for username, rank in ranks.items():
        total_records[username][rank] += 1

    # Store the playoff seed
    playoff_seeds[tuple(ranks.index[:6])] +=1

  0%|          | 0/100 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [61]:
# Stores the similated records in a dataframe and in a csv
records_df = (pd.DataFrame(total_records)
              .transpose()
              .sort_values(by = list(range(1,13)), ascending = False))

records_df.to_csv(f'{YEAR}_{N}_simulated_records.csv', index_label = 'username')

# Stores the simulated playoff seeds in a dataframe and csv
playoffs_df = (pd.DataFrame(playoff_seeds, index = ["Count"])
               .transpose()
               .reset_index()
               .rename(columns={'level_0': 1, 'level_1': 2, 'level_2': 3,
                                'level_3': 4, 'level_4': 5, 'level_5': 6})
               .sort_values(by = 'Count', ascending = False))
playoffs_df.to_csv(f'{YEAR}_{N}_simulated_playoffs.csv')

  0%|          | 0/100 [00:36<?, ?it/s]


KeyboardInterrupt: 

In [61]:
if __name__ == '__main__':
    # Number of times to run the function
    num_iterations = 10

    # Create a partial function with fixed arguments except i
    partial_simulate_schedule = partial(sleepy.simulate_schedule,
                                        all_weeks=all_weeks,
                                        similar_weeks=similar_weeks,
                                        matchups=matchups,
                                        owners=owners,
                                        owners_points=owners_points,
                                        matchup_to_roster_id=matchup_to_roster_id,
                                        total_records_conc=total_records_conc,
                                        playoff_seeds_conc=playoff_seeds_conc)
    
    # Using ProcessPoolExecutor to parallelize updates
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        # Map the function over the range of num_iterations, passing i as the last argument
        executor.map(partial_simulate_schedule, range(num_iterations))

    print(pd.DataFrame(dict(total_records_conc)))

    alecwilson  namebrant  therealfergus  empireyikesback  pacc  \
1            0          0              0                0     0   
2            0          0              0                0     0   
3            0          0              0                0     0   
4            0          0              0                0     0   
5            0          0              0                0     0   
6            0          0              0                0     0   
7            0          0              0                0     0   
8            0          0              0                0     0   
9            0          0              0                0     0   
10           0          0              0                0     0   
11           0          0              0                0     0   
12           0          0              0                0     0   

    tonygordzilla22  mackjyers21  burgertownthicnred  thezirconisdragon  \
1                 0            0                   0 

In [97]:
if __name__ == '__main__':
    num_processes = multiprocessing.cpu_count()
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
        futures = [executor.submit(sleepy.simulate_schedule, all_weeks, similar_weeks, matchups, owners, owners_points, matchup_to_roster_id, i) for i in range(10)]
        for future in concurrent.futures.as_completed(futures):
            print(future.result())

pacc                   1
thezirconisdragon      2
herbietime             3
empireyikesback        4
alecwilson             5
mackjyers21            6
therealfergus          7
burgertownthicnred     8
shakylegs              9
namebrant             10
tonygordzilla22       11
black8yellownation    12
Name: rank, dtype: int64
thezirconisdragon      1
herbietime             2
mackjyers21            3
pacc                   4
empireyikesback        5
alecwilson             6
shakylegs              7
namebrant              8
therealfergus          9
tonygordzilla22       10
burgertownthicnred    11
black8yellownation    12
Name: rank, dtype: int64
herbietime             1
pacc                   2
alecwilson             3
empireyikesback        4
thezirconisdragon      5
therealfergus          6
burgertownthicnred     7
mackjyers21            8
namebrant              9
shakylegs             10
black8yellownation    11
tonygordzilla22       12
Name: rank, dtype: int64
herbietime             1


In [211]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()

Process SpawnProcess-17:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/sleeper/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/sleeper/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/sleeper/lib/python3.11/concurrent/futures/process.py", line 249, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/sleeper/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'is_prime' on <module '__main__' (built-in)>
Process SpawnProcess-18:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/sleeper/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/sleeper/lib/python3.

OSError: handle is closed

In [201]:
# Reads previously stored simulated records
records_df = pd.read_csv(f'{YEAR}_{N}_simulated_records.csv', index_col="username")
records_df = records_df / (N / 100)
records_df

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
herbietime,51.9071,24.6609,12.6393,6.3803,2.8044,1.0866,0.3978,0.0964,0.0245,0.0027,0.0000,0.0000
thezirconisdragon,20.7216,21.7918,21.1837,19.0465,9.0851,4.3249,2.1097,1.0387,0.4521,0.1791,0.0599,0.0069
pacc,12.9685,23.5765,25.8494,20.1825,9.5582,4.4196,2.0366,0.8927,0.3493,0.1283,0.0353,0.0031
alecwilson,12.1560,23.0819,24.2304,20.0035,11.2221,5.5459,2.4291,0.9838,0.2780,0.0607,0.0085,0.0001
empireyikesback,1.0190,2.9865,6.5239,12.8155,22.1379,23.0222,15.4714,9.0484,4.4346,1.8864,0.5803,0.0739
therealfergus,0.7938,2.3112,5.2386,11.1351,22.3924,20.9612,16.0202,10.7515,6.1428,2.9795,1.1170,0.1567
mackjyers21,0.2929,0.9658,2.4152,5.2923,10.2427,15.7720,20.0785,21.3672,12.6290,6.8633,3.3428,0.7383
burgertownthicnred,0.1186,0.5082,1.4703,3.7376,8.5808,15.9720,23.5133,22.1258,13.5556,7.0482,2.8953,0.4743
shakylegs,0.0174,0.0829,0.2860,0.7853,2.0504,4.2744,8.1483,14.8984,26.6940,23.4378,15.1138,4.2113


In [202]:
(records_df * 10000).astype(int)

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
herbietime,519071,246609,126393,63803,28043,10866,3978,964,245,27,0,0
thezirconisdragon,207216,217917,211837,190465,90851,43249,21097,10387,4521,1791,599,69
pacc,129685,235765,258494,201825,95582,44196,20366,8927,3493,1283,353,31
alecwilson,121560,230819,242304,200035,112221,55458,24291,9838,2780,607,85,1
empireyikesback,10189,29865,65239,128155,221378,230222,154714,90484,44346,18864,5803,738
therealfergus,7937,23112,52386,111351,223923,209612,160202,107515,61428,29794,11170,1567
mackjyers21,2929,9658,24152,52923,102426,157720,200784,213672,126290,68633,33428,7383
burgertownthicnred,1186,5082,14703,37376,85808,159720,235133,221258,135556,70482,28953,4743
shakylegs,174,829,2859,7853,20503,42744,81483,148984,266940,234378,151138,42112


In [203]:
# Highlights each user's actual ranking from the season in the simulated dataframe
def bold_cells(row):
    col_to_highlight = season_ranks[row.name]
    return ['color: red' if int(col) == int(col_to_highlight) else '' for col in row.index]

highlighted_df = records_df.astype(str).style.apply(bold_cells , axis = 1)
highlighted_df

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
herbietime,51.9071,24.6609,12.6393,6.3803,2.8044,1.0866,0.3978,0.0964,0.0245,0.0027,0.0,0.0
thezirconisdragon,20.7216,21.7918,21.1837,19.0465,9.0851,4.3249,2.1097,1.0387,0.4521,0.1791,0.0599,0.0069
pacc,12.9685,23.5765,25.8494,20.1825,9.5582,4.4196,2.0366,0.8927,0.3493,0.1283,0.0353,0.0031
alecwilson,12.156,23.0819,24.2304,20.0035,11.2221,5.5459,2.4291,0.9838,0.278,0.0607,0.0085,0.0001
empireyikesback,1.019,2.9865,6.5239,12.8155,22.1379,23.0222,15.4714,9.0484,4.4346,1.8864,0.5803,0.0739
therealfergus,0.7938,2.3112,5.2386,11.1351,22.3924,20.9612,16.0202,10.7515,6.1428,2.9795,1.117,0.1567
mackjyers21,0.2929,0.9658,2.4152,5.2923,10.2427,15.772,20.0785,21.3672,12.629,6.8633,3.3428,0.7383
burgertownthicnred,0.1186,0.5082,1.4703,3.7376,8.5808,15.972,23.5133,22.1258,13.5556,7.0482,2.8953,0.4743
shakylegs,0.0174,0.0829,0.286,0.7853,2.0504,4.2744,8.1483,14.8984,26.694,23.4378,15.1138,4.2113


In [208]:
def get_prob(username, rank, type = "equal"):
    """Gets the probability of a user ranking equal to, worse, or better than 
    they actually did. Probabilites are taken from the simulated rankings.

    Args:
        username (str): the username to get the probability ranking
        rank (int): the rank to determinge
        type (str, optional): Can be "equal", "worse", or "better". For each 
        option. Defaults to "equal".

    Returns:
        str: The function returns the probability of the given user ranking 
        equal/better/or worse than the passed rank. Probabilities are taken from
        the simulated records dataframe.
    """
    user_ix = records_df.index.get_loc(username)
    if type == "equal":
        return records_df.iloc[user_ix, rank-1]
    elif type == "worse":
        return records_df.iloc[user_ix, rank:].sum()
    elif type == "better":
        return records_df.iloc[user_ix, :rank-1].sum()
    elif type == "playoff":
        return records_df.iloc[user_ix, :6].sum()

# Gets the probability dictionaries of worse/equal/better for each user
probs_worse = {username: get_prob(username, rank, type = "worse") for (username, rank) in season_ranks.items()}
probs_better = {username: get_prob(username, rank, type = "better") for (username, rank) in season_ranks.items()}
probs_equal = {username: get_prob(username, rank, type = "equal") for (username, rank) in season_ranks.items()}
probs_playoff = {username: get_prob(username, rank, type = "playoff") for (username, rank) in season_ranks.items()}

# Stores the probabilites of each user's ranking for the season
season_probs = (pd.DataFrame(season_ranks)
                .assign(Better = pd.Series(probs_better))
                .assign(Equal = pd.Series(probs_equal))
                .assign(Worse = pd.Series(probs_worse))
                .assign(Playoff = pd.Series(probs_playoff))
                .reset_index(names="username")
                .set_index("rank")
                .round(2))
season_probs

,username,Better,Equal,Worse,Playoff
rank,,,,,
1,pacc,0.00,12.97,87.03,96.55
2,thezirconisdragon,20.72,21.79,57.49,96.15
3,herbietime,76.57,12.64,10.79,99.48
4,alecwilson,59.47,20.00,20.53,96.24
5,empireyikesback,23.34,22.14,54.52,68.50
6,burgertownthicnred,14.42,15.97,69.61,30.39
7,therealfergus,62.83,16.02,21.15,62.83
8,mackjyers21,55.06,21.37,23.57,34.98
9,shakylegs,30.54,26.69,42.76,7.50


In [214]:
# Plots the season probabilities as a stacked bar chart
season_probs_melt = pd.melt(season_probs, id_vars=['username', "Playoff"], var_name=' ', value_name='Chance %')
fig = px.bar(
        season_probs_melt, 
        x='username', 
        y='Chance %', 
        color = ' ', 
        title = f'Chance of Ranking: Worse Than / Equal To / Better Than Actual Ranking for {YEAR} Season',
        color_discrete_sequence = px.colors.qualitative.D3[:3][::-1],
        )

fig.update_layout(width=1200,
                  height=600,
                  xaxis_title = None,
                  title_x = 0.5,
                  font=dict(size=14),
                  margin=dict(t=70, b=75, l=100, r=50),
                  bargap=0.5,
                  legend = dict(title = None,
                                font = dict(size=20))
                 )

In [192]:
playoffs_df = (pd.DataFrame(playoff_seeds, index = ["Count"])
               .transpose()
               .reset_index()
               .rename(columns={'level_0': 1, 'level_1': 2, 'level_2': 3,
                                'level_3': 4, 'level_4': 5, 'level_5': 6}))

playoffs_df = playoffs_df[playoffs_df["Count"] > 0]

In [196]:
playoffs_df_prob = pd.DataFrame(playoffs_df)

def better_equal(username, rank):
    return get_prob(username, rank, type = "better") + get_prob(username, rank, type = "equal")

for i in range(1,7):
    playoffs_df_prob[i] = playoffs_df[i].apply(lambda x: better_equal(x,i))

In [197]:
idx_unlikely = playoffs_df_prob.iloc[:,:6].sum(axis = 1).idxmin()

In [198]:
unlikely_playoff_seed = (pd.concat([playoffs_df.loc[451170,:6].rename("Username"),
                                    playoffs_df_prob.loc[451170,:6].rename("Prob >=")],
                                      axis = 1)
                            .rename(columns={f'{idx_unlikely}':'as'}))
unlikely_playoff_seed

,Username,Prob >=
1,thezirconisdragon,20.7216
2,namebrant,0.0367
3,mackjyers21,3.6739
4,therealfergus,19.4787
5,burgertownthicnred,14.4155
6,shakylegs,7.4964


In [202]:
playoffs_df[playoffs_df[1] == 'tonygordzilla22']

,1,2,3,4,5,6,Count
324592,tonygordzilla22,herbietime,pacc,alecwilson,empireyikesback,mackjyers21,1
